## Contents
1. <a href='#Have people in Estonia stopped marrying'>Have people in Estonia stopped marrying?</a>
2. <a href='#Did the age of first marriage grew?'>Did the age of first marriage grew?</a>
3. <a href='#What is the most popular time of the year to get married?'>What is the most popular time of the year to get married?</a>
4. <a href='#Love after first marriage. How often do Estonians remarry?'>Love after first marriage. How often do Estonians remarry?</a>
5. <a href='#Babies come before or after marriage?'>Babies come before or after marriage?</a>
6. <a href='#Maps of comparison with European countries'>Maps of comparison with European countries</a>

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from sklearn.linear_model import LinearRegression

import chart_studio
import plotly.graph_objects as go
import plotly.figure_factory as ff
from _plotly_future_ import v4_subplots
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import plotly.graph_objects as go
import chart_studio.tools as tls
import plotly.express as px

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

If you're interested in exporting your visualizations, I recommend you to register at https://chart-studio.plot.ly. Then just enter your username and api key in the cell bellow. Deactivate the cell otherwise.
Sidenote, the interactive map plots for European countries through time also require the authentification with chart-studio, at least, in the following version of the code.

In [95]:
username = '' # your username
api_key = '' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [72]:
path = 'Data/'

In [73]:
# Creatiing annual population of adults for a better comparison
df_annual_population = pd.read_csv(path + 'RV0211_Mean_annual_population_by_sex_and_age_group.csv')
df_annual_population.set_index("Age group", inplace=True)
df_annual_population = df_annual_population.loc[['45-49', '15-19', '75-79', '30-34', 
       '60-64', '25-29', '80-84', '40-44', '35-39', '65-69', '20-24', '85 and older', '55-59', '50-54', '70-74']]
df_annual_population= df_annual_population.loc[df_annual_population['Sex'] == 'Males and females']
df_annual_population.reset_index(inplace=True)
df_annual_population_adults = df_annual_population[['TIME', 'Value']].groupby(by = 'TIME').sum()
df_annual_population_adults.reset_index(inplace=True)
df_annual_population_adults.rename(columns={'Value':'Mean Annual Population Adults'}, inplace=True)

<a id='Have people in Estonia stopped marrying?'></a>
## Have people in Estonia stopped marrying?

In [74]:
df_marriage_1 = pd.read_excel(path +'RV047_Marriages_and_divorces.xlsx')

df_marriage_1 = df_marriage_1[df_marriage_1['TIME'] >= 1970]

df_marriage_1['Marriages per 1,000 population'] = df_marriage_1['Marriages per 1,000 population'].apply(lambda x: float(x))
df_marriage_1['Marriages'] = df_marriage_1['Marriages'].apply(lambda x: int(x))
df_marriage_1['Divorces'] = df_marriage_1['Divorces'].apply(lambda x: int(x))

# Marriages per 1,000 population is a good metric but it can be more precise. 
# Since not everyone, age-wise, can get married, it makes sense to look at the rate per adults
df_marriage_1 = df_marriage_1.merge(df_annual_population_adults, on = 'TIME')
df_marriage_1['Marriages per 1,000 adults population'] = df_marriage_1['Marriages'] / df_marriage_1['Mean Annual Population Adults'] * 1000

df_marriage_1.head(2)

,TIME,Marriages,Divorces,"Marriages per 1,000 population","Divorces per 1,000 population",Mean Annual Population Adults,"Marriages per 1,000 adults population"
0,1970,12373,4379,9.10,3.22,1059235,11.681072
1,1971,12514,4354,9.09,3.16,1072641,11.666531


In [75]:
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df_marriage_1['TIME'], y=df_marriage_1['Marriages per 1,000 adults population'],
                    mode='lines+markers',
                    name='Marriages per 1,000 adults',
                    line_color='rgb(15,62,40)'),
             secondary_y=True,)

fig.add_trace(go.Scatter(x=df_marriage_1['TIME'], y=df_marriage_1['Marriages'],
                    mode='lines+markers',
                    name='Marriages',
                    line_color='rgb(255, 69, 0)'))

fig.update_yaxes(title_text="Marriages per 1,000 adults", secondary_y=True, range=[0, 12])
fig.update_yaxes(title_text="Number of Marriages", secondary_y=False, range=[0, 14000])


fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        type = 'category',
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=20,
        r=10,
        t=110,
    ),
    showlegend=True,
    legend=dict(x=0.6, y=1.),
    plot_bgcolor='white',
    title = 'Marriages in Estonia through years'
)

fig.show()

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.


<a id='Did the age of first marriage grew?'></a>
## Did the age of first marriage grew? 


In [76]:
df_marriage_2 = pd.read_excel(path +'RV048_Total_first_marriage_rate_and_mean_age_at_first_marriage_by_sex.xlsx')
df_marriage_2.head(2)

,TIME,Total first marriage rate Males,Total first marriage rate Females,Mean age at first marriage – Males,Mean age at first marriage – Females
0,1970,0.99,1.04,25.6,23.5
1,1971,0.98,1.02,25.4,23.5


In [77]:
fig = go.Figure()
fig = make_subplots()

fig.add_trace(go.Scatter(x=df_marriage_2['TIME'], y=df_marriage_2['Mean age at first marriage – Males'],
                    mode='lines+markers',
                    name='Mean age at first marriage – Males',
                    line_color='rgb(15,62,40)'))

fig.add_trace(go.Scatter(x=df_marriage_2['TIME'], y=df_marriage_2['Mean age at first marriage – Females'],
                    mode='lines+markers',
                    name='Mean age at first marriage – Females',
                    line_color='rgb(53,209,135)'))

fig.update_yaxes(title_text="Age", secondary_y=False)


fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        type = 'category',
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=20,
        r=10,
        t=110,
    ),
    showlegend=True,
    legend=dict(x=0.04, y=1.),
    plot_bgcolor='white',
    title = 'Mean age at first marriage for Males and Females'
)

fig.show()

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.


In [78]:
#Linear regression for predicting how the mean age at first marriage for women is  going to change over then next 12 years
x = df_marriage_2[df_marriage_2['TIME']>=1992]['TIME'].values.reshape(-1, 1)
y = df_marriage_2[df_marriage_2['TIME']>=1992]['Mean age at first marriage – Females'].values.reshape(-1, 1)
reg = LinearRegression(normalize = True).fit(x, y)
reg.score(x, y)
future_x = np.array([range(2019, 2030)]).reshape(-1, 1)
future = reg.predict(future_x)

my_age = list(range(0, 40))

fig = go.Figure()
fig = make_subplots()

fig.add_trace(go.Scatter(x=df_marriage_2[df_marriage_2['TIME']>=1993]['TIME'], y=df_marriage_2[df_marriage_2['TIME']>=1993]['Mean age at first marriage – Females'],
                    mode='lines',
                    name='Reality: Mean age at first marriage – Females',
                    line_color='rgb(15,62,40)'))

fig.add_trace(go.Scatter(x=pd.Series(future_x.flatten()), y=pd.Series(future.flatten()),
                    mode='lines',
                    name='Projection: Mean age at first marriage – Females',
                    line_color='rgb(53,209,135)'))

fig.add_trace(go.Scatter(x=df_marriage_2[df_marriage_2['TIME']>=1993]['TIME'].append(pd.Series(future_x.flatten())), y=my_age,
                    mode='lines+markers',
                    name='My age',
                    line_color='rgb(23,109,135)'))



fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        type = 'category',
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=20,
        r=10,
        t=110,
    ),
    showlegend=True,
    legend=dict(x=0.04, y=1.2),
    plot_bgcolor='white'
)

fig.show()

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.


<a id='What is the most popular time of the year to get married?'></a>

## What is the most popular time of the year to get married?

In [79]:
df_marriage_3 = pd.read_excel(path +'PO262_MARRIAGES_by_Year_and_Month_of_contracting_the_marriage.xlsx')


In [80]:
df_marriage_3['Total'] = df_marriage_3.iloc[:, 1:].sum(axis = 1)
df_marriage_4 = df_marriage_3[['TIME','Total']]
df_marriage_3.head(2)


,TIME,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,1970,779,654,845,900,909,1014,1265,1493,1208,1139,955,1212,12373
1,1971,758,641,904,968,896,1094,1495,1394,1165,1083,927,1189,12514


In [81]:
# Changing the absolute values to pecentage to get a more clear representation on a heatmap

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December']

for i in months:
    df_marriage_3[i + '_percent'] = df_marriage_3[i] / df_marriage_3['Total']
df_marriage_3 = df_marriage_3.drop(columns = months).drop(columns =  'Total')
df_marriage_3.head(2)

,TIME,January_percent,February_percent,March_percent,April_percent,May_percent,June_percent,July_percent,August_percent,September_percent,October_percent,November_percent,December_percent
0,1970,0.062960,0.052857,0.068294,0.072739,0.073466,0.081953,0.102239,0.120666,0.097632,0.092055,0.077184,0.097955
1,1971,0.060572,0.051223,0.072239,0.077353,0.071600,0.087422,0.119466,0.111395,0.093096,0.086543,0.074077,0.095014


In [82]:
fig = go.Figure(data=go.Heatmap(
                   z=df_marriage_3.transpose().iloc[1:, ]*100,
                   x=df_marriage_3['TIME'],
                   y=months,
    colorbar=dict(title="Popularity", 
                  ticktext = ['low', 'high'], 
                  tickvals= [0,1],
                  tickformat = '.0%',
                  x = 1.0),

    colorscale=[
        [0.0, "rgb(254, 254, 254)"],
        [1.0, 'rgb(255, 69, 0)']
    ]))


fig.update_layout(
   
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            font=dict(
                family="Arial",
                size=18,
                color="#7f7f7f"
            ) 
        ), type="category"
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            font=dict(
                family="Arial",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    
    title = "Months' popularity for marriage"

)


fig.show()

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.

In [83]:
df_marriage_3['Summer weddings'] = df_marriage_3['June_percent'] +df_marriage_3['July_percent']+ df_marriage_3['August_percent']
df_marriage_3['Winter weddings'] = df_marriage_3['December_percent'] +df_marriage_3['January_percent']+ df_marriage_3['February_percent']
df_marriage_3['Spring weddings'] = df_marriage_3['March_percent'] +df_marriage_3['April_percent']+ df_marriage_3['May_percent']
df_marriage_3['Autumn weddings'] = df_marriage_3['September_percent'] +df_marriage_3['October_percent']+ df_marriage_3['November_percent']

fig = go.Figure()
fig = make_subplots()

fig.add_trace(go.Scatter(x=df_marriage_3['TIME'], y=df_marriage_3['Summer weddings'],
                    mode='lines+markers',
                    name='Summer weddings popularity',
                    line_color='rgb(255,202,39)'))

fig.add_trace(go.Scatter(x=df_marriage_3['TIME'], y=df_marriage_3['Winter weddings'],
                    mode='lines+markers',
                    name='Winter weddings popularity',
                    line_color='rgb(117,205,216)'))

fig.add_trace(go.Scatter(x=df_marriage_3['TIME'], y=df_marriage_3['Spring weddings'],
                    mode='lines+markers',
                    name='Spring weddings popularity',
                    line_color='rgb(34,139,34)'))

fig.add_trace(go.Scatter(x=df_marriage_3['TIME'], y=df_marriage_3['Autumn weddings'],
                    mode='lines+markers',
                    name='Autumn weddings popularity',
                    line_color='rgb(255, 69, 0)'))


# fig.update_yaxes(title_text="Marriages per 1,000 adults", secondary_y=True)
# fig.update_yaxes(title_text="Number of Marriages", secondary_y=False)
fig.update_yaxes(title_text="Percentage of weddings", secondary_y=False, range=[0, 0.5])

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        type = 'category',
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        tickformat = '.0%',
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=20,
        r=10,
        t=110,
    ),
    showlegend=True,
    legend=dict(x=0.04, y=1.),
    plot_bgcolor='white',
    title = 'Change in popularity of seasons for a wedding'
)

fig.show()

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.

In [84]:
x = df_marriage_3['TIME']
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x, y=100*df_marriage_3['Winter weddings'],
    mode='lines',
    line=dict(width=0.5, color='rgb(117,205,216)'),
    stackgroup='one',
    groupnorm='percent', # sets the normalization for the sum of the stackgroup,
    name = 'Winter weddings'
))
fig.add_trace(go.Scatter(
    x=x, y=100*(df_marriage_3['Spring weddings']),
    mode='lines',
    line=dict(width=0.5, color='rgb(34,139,34)'),
    stackgroup='one',
    name = 'Spring weddings'

))
fig.add_trace(go.Scatter(
    x=x, y=100*(df_marriage_3['Autumn weddings']),
    mode='lines',
    line=dict(width=0.5, color='rgb(255, 69, 0)'),
    stackgroup='one',
    name = 'Autumn weddings'
))
fig.add_trace(go.Scatter(
    x=x, y=100*(df_marriage_3['Summer weddings']),
    mode='lines',
    line=dict(width=0.5, color='rgb(255,202,39)'),
    stackgroup='one',
    name = 'Summer weddings'
))

fig.update_layout(
    showlegend=True,
    xaxis_type='category',
        plot_bgcolor='white',

    yaxis=dict(
        type='linear',
        range=[1, 100],
        ticksuffix='%'))

fig.show()

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.

<a id='Love after first marriage. How often do Estonians remarry?'></a>

## Love after first marriage. How often do Estonians remarry?

In [85]:
df_marriage_5 = pd.read_excel(path +'RV27_Newly_married_persons_by_sex.xls')
df_marriage_5['First married Totals'] = df_marriage_5['First married persons -Males'] + df_marriage_5['First married persons – Females'] 
df_marriage_5['Remarried Totals'] = df_marriage_5['Remarried persons – Males'] + df_marriage_5['Remarried persons – Females'] 
df_marriage_5['Remarriage Percentage'] = df_marriage_5['Remarried Totals']  / (df_marriage_5['First married Totals'] + df_marriage_5['Remarried Totals'])
df_marriage_5.head(1)

,TIME,First married persons -Males,First married persons – Females,Remarried persons – Males,Remarried persons – Females,First married Totals,Remarried Totals,Remarriage Percentage
0,1970,10216,10333,2157,2040,20549,4197,0.169603


In [86]:

fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
    
fig.add_trace(
    go.Bar(name='Marrying for the first time', x=df_marriage_5['TIME'], y=df_marriage_5['First married Totals'],
    marker=dict(color='rgb(117,205,216)')                    
    ))

fig.add_trace(
    go.Bar(name='Remarrying', x=df_marriage_5['TIME'], y=df_marriage_5['Remarried Totals'],
    marker=dict(color='rgb(255, 69, 0)'
    )))



fig.add_trace(go.Scatter(x=df_marriage_5['TIME'], y=df_marriage_5['Remarriage Percentage'],
                    mode='lines+markers',
                    name='Remarriage Percentage',
                    line_color='rgb(15,62,40)'),
             secondary_y=True)

fig.update_yaxes(title_text="% of remarriages", secondary_y=True, 
        tickformat = '.0%', 
        range=[0, 0.4],)


# Change the bar mode
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        type = 'category',
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=20,
        r=10,
        t=110,
    ),
    showlegend=True,
    legend=dict(x=0.7, y=1.),
    plot_bgcolor='white',
    title = 'First time married vs remarried persons'
)


fig.show()

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.

<a id='Babies come before or after marriage?'></a>

## Babies come before or after marriage?

In [87]:
df_children1 = pd.read_excel(path +'Eurostat_Mean_age_of_women_at_childbirth_and_at_birth_of_first_child.xlsx')
df_children11 = df_children1[df_children1.iloc[:, 0]  == 'Estonia'] 
df_children11.head(1)
#Source : https://ec.europa.eu/eurostat/tgm/table.do?tab=table&plugin=1&language=en&pcode=tps00017

,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
9,Estonia,25.2,25.3,25.7,26,26.3,26.4,26.5,26.5,26.6,27.2,27.5,27.7


In [88]:
fig = go.Figure()
fig = make_subplots()

fig.add_trace(go.Scatter(x=np.array(df_children11.columns[1:]), y=df_children11.iloc[0:,1:].values[0].astype(float),
                    mode='lines+markers',
                    name='Mean age at birth of first child - Females',
                    line_color='rgb(15,62,40)'))

fig.add_trace(go.Scatter(x=df_marriage_2[df_marriage_2['TIME'] >=2006][df_marriage_2['TIME'] <=2017]['TIME'], 
                         y=df_marriage_2[df_marriage_2['TIME'] >=2006][df_marriage_2['TIME'] <=2017]['Mean age at first marriage – Females'],
                    mode='lines+markers',
                    name='Mean age at first marriage – Females',
                    line_color='rgb(227, 116, 131)'))


fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        type = 'category',
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=20,
        r=10,
        t=110,
    ),
    showlegend=True,
    legend=dict(x=0.04, y=1.),
    plot_bgcolor='white',
    title = 'Comparison of mean age at birth of first child and at first marriage for Females'
)

fig.show()

/home/polina/Development/eesti-stats/env/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/home/polina/Development/eesti-stats/env/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



<a id='Maps of comparison with European countries'></a>

## Maps of comparison with European countries
Kudos to Julia Poncela-Casasnovas whose post helped me to create these maps: https://amaral.northwestern.edu/blog/step-step-how-plot-map-slider-represent-time-evolu

### Mean age at first marriage by sex males

In [96]:
df_marriage6 = pd.read_excel(path + 'TPS00014_Mean_age_at_first_marriage_by_sex_males.xlsx')
df_marriage6 = df_marriage6.dropna(how='all', axis=1)
df_marriage6 = df_marriage6.drop(columns = ['Unnamed: 8', 'Unnamed: 12', 'Unnamed: 14']) 
df_marriage6 = df_marriage6.set_index('Country')
df_marriage6 = df_marriage6.iloc[3:, :].drop(['France (metropolitan)', 'Germany (until 1990 former territory of the FRG)'], axis=0)
df_marriage6 = df_marriage6.reset_index()
df_marriage6.head(2)

,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Belgium,30.9,31,31.4,31.7,31.7,:,:,:,:,:,:,33.5
1,Bulgaria,29.2,29.1,29.2,29.4,29.8,29.9,:,30,30.3,30.4,30.5,30.6


In [97]:
### create empty list for data object:    
data_slider = []


for year in list(df_marriage6.columns[1:]):
    df_marriage6_year = df_marriage6[year]
    data_one_year = dict(
                        type='choropleth',
                        locations = df_marriage6['Country'],
                        z=df_marriage6_year.replace(":", np.nan).astype(float),
                         locationmode = 'country names',
                         text = 'Mean age',
                        )

    data_slider.append(data_one_year)  # I add the dictionary to the list of dictionaries for the slider


In [98]:
steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2006)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)


##  I create the 'sliders' object from the 'steps' 

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]  


layout = dict(geo=dict(scope='europe'),title_text = 'Mean age at first marriage by sex - Males, Europe',
              autosize=True,
              sliders=sliders)

    
    
# I create the figure object:

fig = dict(data=data_slider, layout=layout) 


# to plot in the notebook

py.iplot(fig)

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.

### Mean age at first marriage by sex females

In [99]:
df_marriage7 = pd.read_excel(path + 'TPS00014_Mean_age_at_first_marriage_by_sex_females.xlsx')
df_marriage7 = df_marriage7.dropna(how='all', axis=1)
df_marriage7 = df_marriage7.drop(columns = ['Unnamed: 8', 'Unnamed: 12', 'Unnamed: 14'])
df_marriage7 = df_marriage7.set_index('Country')
df_marriage7 = df_marriage7.iloc[3:, :].drop(['France (metropolitan)', 'Germany (until 1990 former territory of the FRG)'], axis=0)
df_marriage7 = df_marriage7.reset_index()


In [100]:
### create empty list for data object:    
data_slider = []


for year in list(df_marriage7.columns[1:]):
    df_marriage7_year = df_marriage7[year]
    data_one_year = dict(
                        type='choropleth',
                        locations = df_marriage7['Country'],
                        z=df_marriage7_year.replace(":", np.nan).astype(float),
                         locationmode = 'country names',
                         text = 'Mean age',
                        )

    data_slider.append(data_one_year)  # I add the dictionary to the list of dictionaries for the slider


In [101]:
steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2006)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)


##  I create the 'sliders' object from the 'steps' 

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]  


layout = dict(geo=dict(scope='europe'),title_text = 'Mean age at first marriage by sex - Females, Europe',
              autosize=True,
              sliders=sliders)

    
    
# I create the figure object:

fig = dict(data=data_slider, layout=layout) 


# to plot in the notebook

py.iplot(fig)

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.


### Mean age at birth of first child by sex females

In [102]:
df_children12 = df_children1.set_index('Country')
df_children12 = df_children12.iloc[4:, :].drop(['France (metropolitan)'], axis=0)
df_children12 = df_children12.reset_index()
df_children12.head(3)

,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Belgium,27.7,27.8,27.8,28.1,28,28.1,28.2,28.5,28.5,28.7,28.8,29
1,Bulgaria,24.9,25,25,25.2,25.6,25.7,25.6,25.7,25.8,26,26,26.1
2,Czechia,26.9,27.1,27.3,27.4,27.6,27.8,27.9,28.1,28.1,28.2,28.2,28.2


In [103]:
### create empty list for data object:    

data_slider = []


for year in list(df_children12.columns[1:-1]):
    df_children12_year = df_children12[year]
    data_one_year = dict(
                        type='choropleth',
                        locations = df_children12['Country'],
                        z=df_children12_year.replace(":", np.nan).astype(float),
                         locationmode = 'country names',
                         text = 'Mean age',
                        )

    data_slider.append(data_one_year)  # I add the dictionary to the list of dictionaries for the slider


In [104]:
steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2006)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)


##  I create the 'sliders' object from the 'steps' 

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]  


layout = dict(geo=dict(scope='europe'),title_text = 'Mean age at birth of first child - Females, Europe',
              autosize=True,
              sliders=sliders)

    
    
# I create the figure object:

fig = dict(data=data_slider, layout=layout) 


# to plot in the notebook

py.iplot(fig)

# py.plot(fig, filename = '', auto_open=True) - uncomment and inpute your filename if you want to export to chart-studio.plot.ly.
